In [3]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
inf = float('inf')

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

In [851]:
f = open("in.txt").read().split("\n")
lines = [list(line) for line in f]
rows = len(lines)
cols = len(lines[0])
rows, cols

(131, 131)

In [72]:
N = (-1,0)
S = (1,0)
E = (0,1)
W = (0,-1)     


In [390]:
gardensize[(-2, 0)]

[2, 4, 6, 8, 14, 14, 17, 20, 26, 27, 33, 34, 39, 38]

In [399]:
gardenopentile[(-2,0)]

((10, 0), False, True)

In [397]:
gardenopentile[gardenloc]

((10, 0), False, True)

In [852]:
file = 'in.txt'
f = open(file).read().split("\n")
lines = [list(line) for line in f]
rows = len(lines)
cols = len(lines[0])
rows, cols
# grid
grid, neigh = gridneigh(file, to_dict=True, parser=lambda x: [(ch) for ch in x])
# need = 64
maxx = 7717
other = 7693
# maxx = 42
# other = 39

borders = defaultdict(list)
for k,v in grid.items():
    for wind in [N, S, E, W]:
        new = k[0] + wind[0], k[1] + wind[1]
        if new not in grid:
            new = new[0] % rows, new[1] % cols
            assert new in grid
            if grid[new] != '#':
                borders[k].append((wind, new))



start = [k for k,v in grid.items() if v == 'S'].pop()
valids = {k for k,v in grid.items() if v != '#'}



on = defaultdict(lambda: [0,0])
opengarden = {} # coordinates

gardenopentime = {(0,0):1}
gardenopentime[(0,0)] = 0

opened = [(0,0)]
gardensize = defaultdict(list)
gardensize[0,0].append(1)
gardenopentile = {}
gardenopentile[(0,0)] = start, 0, True, True  #, True, True, False, False, False, False

made = defaultdict(list) # r,c identifier and what they made
closedgarden = dict() # coordinates with a stepmoment
gardens = {(0,0): {start}} # dict with coordinates and a set of cells per garden
need = 1200
careful = 0
makers = defaultdict(int)
for step in range(1,need+1):
    
    newgardens = defaultdict(set)
    oldgarden = gardens.copy()
    for coor, garden in gardens.items():
        for tile in garden:        
            # print(tile)
            for n in neigh[tile]:
                if n in valids:
                    newgardens[coor].add(n)
            for wind, newcoor in borders[tile]:
                # for all bordertiles
                gardenloc = coor[0] + wind[0], coor[1] + wind[1]
                if gardenloc not in closedgarden: # else ignore
                    if gardenloc not in gardens:
                        makers[gardenloc] = step
                        # we are making a new garden
                        newtile = newcoor, abs(gardenloc[0])+abs(gardenloc[1]), gardenloc[0] == 0, gardenloc[1] == 0 #, gardenloc[0] == 1, gardenloc[1] == 1,gardenloc[0] == -1, gardenloc[1] == -1
                        if gardenloc not in gardenopentile:
                            gardenopentime[gardenloc] = step
                            gardenopentile[gardenloc] = newtile
                            made[coor].append((step - gardenopentime[coor], newtile))
                            # print(coor, gardenloc, step, gardenopentile[coor], made[coor])
                        # check right now assumption that will be made only once
                        
                        
                        # if gardenopentile[gardenloc] == ((10, 10), False, True) or gardenopentile[gardenloc] == ((10, 0), False, True):
                        #     print('hoi',step, gardenloc,gardenopentile[gardenloc])
                        #     print('made', coor, tile, gardenloc, (step - gardenopentime[coor], gardenopentile[gardenloc]))
                        # if newcoor == (0,0):
                            opened.append(gardenloc)
                        
                        
                    else:
                        if all(n2 not in oldgarden[gardenloc] for n2 in neigh[newcoor]):
                            # print(step, makers[gardenloc], newcoor, oldgarden[gardenloc])
                            careful += 1
                    
                    # print(newcoor)
                    newgardens[gardenloc].add(newcoor)
    gardens = newgardens
    for coor in list(gardens.keys()):
        if len(gardens[coor]) == maxx:
            closedgarden[coor] = step
            del gardens[coor]
    for coor, garden in gardens.items():
        lifetime = step - gardenopentime[coor]
        if lifetime == len(gardensize[coor]):
            gardensize[coor].append(len(garden))
        else:
            print('errro')


    if step in [6,10,50,64, 100,500,1000,5000] or not step % 1000000:
        opentiles = sum([len(v) for k,v in gardens.items()])
        closedtilesmaxx = sum([maxx for k,v in closedgarden.items() if v % 2 == step % 2])
        closedtilesother = sum([other for k,v in closedgarden.items() if v % 2 != step % 2])
        print(step, sum([opentiles, closedtilesmaxx, closedtilesother]), opentiles, closedtilesmaxx, closedtilesother)
        print('open gardens', len(gardens), 'closed gardens', len(closedgarden))
    # print(gardens.keys())
    


6 41 41 0 0
open gardens 1 closed gardens 0
10 103 103 0 0
open gardens 1 closed gardens 0
50 2301 2301 0 0
open gardens 1 closed gardens 0
64 3782 3782 0 0
open gardens 1 closed gardens 0
100 9175 9175 0 0
open gardens 5 closed gardens 0
500 225441 125336 30868 69237
open gardens 28 closed gardens 13
1000 900187 245418 277812 376957
open gardens 60 closed gardens 85


In [853]:
len(closedgarden)


145

In [854]:
# check what gardens are made from a certain gardentype (if they always make the same)
# gardenopentime, gardensize, gardenopentile
baby = defaultdict(lambda: defaultdict(list))
for tile in sorted(set(gardenopentile.values())):
    similar = defaultdict(list)

    
    found = defaultdict(list)
    for k, size in made.items():
        if gardenopentile[k] == tile and 500 < gardenopentime[k] < 800:
            similar[(tuple(sorted(size)))].append(k)
    print('\n', tile)
    for children, coor in similar.items():
        print(tile, 'c', children, 'coor', coor)


 ((0, 0), 2, False, False)

 ((0, 0), 3, False, False)

 ((0, 0), 4, False, False)

 ((0, 0), 5, False, False)
((0, 0), 5, False, False) c ((131, ((0, 0), 6, False, False)),) coor [(4, 1), (3, 2), (2, 3), (1, 4)]

 ((0, 0), 6, False, False)
((0, 0), 6, False, False) c ((131, ((0, 0), 7, False, False)),) coor [(5, 1), (4, 2), (3, 3), (2, 4), (1, 5)]

 ((0, 0), 7, False, False)
((0, 0), 7, False, False) c ((131, ((0, 0), 8, False, False)),) coor [(6, 1), (5, 2), (4, 3), (3, 4), (2, 5), (1, 6)]

 ((0, 0), 8, False, False)

 ((0, 0), 9, False, False)

 ((0, 0), 10, False, False)

 ((0, 65), 1, False, True)

 ((0, 65), 2, False, True)

 ((0, 65), 3, False, True)

 ((0, 65), 4, False, True)

 ((0, 65), 5, False, True)
((0, 65), 5, False, True) c ((66, ((0, 0), 6, False, False)), (131, ((0, 65), 6, False, True))) coor [(5, 0)]

 ((0, 65), 6, False, True)
((0, 65), 6, False, True) c ((66, ((0, 0), 7, False, False)), (131, ((0, 65), 7, False, True))) coor [(6, 0)]

 ((0, 65), 7, False, True)



In [855]:
baby = defaultdict(lambda: defaultdict(list))
for coor, children in made.items():
    tile = gardenopentile[coor]
    if 250 < gardenopentime[coor] < 600:
        if tile in baby and baby[tile] != children:
            print(baby[tile], children)
        baby[tile] = children
        


In [856]:

# check how quickly they the garden types grow, if its always  the same

# gardenopentime, gardensize, gardenopentile
for tile in sorted(set(gardenopentile.values())):
    similar = set()
    found = defaultdict(list)
    for k, size in gardensize.items():
        gardenopentile[k]
        if gardenopentile[k] == tile  and max(size)>37 and gardenopentime[k] < 400:
            similar.add(tuple(size))
            found[tuple(size)].append((k, gardenopentime[k]))
    print('\n', tile)
    for r in similar:
        print(r,found[r])


 ((0, 0), 2, False, False)
(1, 2, 4, 6, 9, 12, 15, 19, 23, 27, 31, 37, 44, 51, 57, 66, 73, 81, 91, 98, 110, 115, 129, 139, 151, 164, 171, 187, 195, 214, 222, 243, 254, 274, 285, 306, 320, 342, 353, 378, 391, 417, 428, 455, 465, 498, 506, 542, 545, 585, 593, 635, 641, 687, 692, 737, 746, 793, 798, 848, 852, 904, 914, 968, 980, 1034, 1047, 1102, 1116, 1166, 1179, 1232, 1244, 1297, 1309, 1366, 1370, 1436, 1444, 1508, 1511, 1577, 1588, 1642, 1661, 1718, 1744, 1792, 1829, 1877, 1906, 1959, 1986, 2042, 2076, 2131, 2165, 2220, 2256, 2308, 2344, 2405, 2434, 2499, 2526, 2591, 2617, 2690, 2711, 2792, 2806, 2892, 2907, 2996, 3007, 3102, 3114, 3213, 3217, 3320, 3324, 3427, 3430, 3535, 3538, 3645, 3650, 3760, 3765, 3877, 3889, 3993, 4004, 4103, 4119, 4214, 4228, 4325, 4333, 4429, 4445, 4540, 4555, 4643, 4664, 4747, 4773, 4855, 4870, 4955, 4970, 5047, 5068, 5140, 5162, 5234, 5256, 5327, 5351, 5423, 5437, 5512, 5515, 5590, 5596, 5677, 5685, 5769, 5779, 5855, 5864, 5936, 5949, 6018, 6025, 6093, 6100,

In [857]:
gardengrowth = defaultdict(list)
for tile in sorted(set(gardenopentile.values()), key=lambda x: x[1]):
    if tile[1] > 20:
        break
    print('\n', tile)
    for k,v in gardensize.items():
        if gardenopentile[k] == tile:
            print(k,gardenopentime[k], v)
            
            gardengrowth[tile] = max(gardengrowth[tile], v, key= lambda x: len(x))
gardengrowth


 ((65, 65), 0, True, True)
(0, 0) 0 [1, 4, 9, 14, 24, 32, 41, 54, 67, 90, 103, 128, 148, 179, 198, 233, 259, 296, 328, 370, 400, 445, 469, 526, 547, 618, 635, 714, 732, 817, 837, 930, 949, 1041, 1070, 1169, 1212, 1303, 1344, 1433, 1480, 1576, 1634, 1730, 1785, 1874, 1949, 2040, 2122, 2210, 2301, 2383, 2477, 2572, 2665, 2770, 2858, 2971, 3058, 3175, 3267, 3392, 3517, 3652, 3782, 3916, 4042, 4172, 4294, 4394, 4509, 4599, 4710, 4796, 4913, 4996, 5118, 5195, 5301, 5380, 5485, 5559, 5651, 5731, 5811, 5897, 5974, 6062, 6130, 6209, 6269, 6348, 6400, 6493, 6541, 6625, 6666, 6748, 6778, 6857, 6889, 6958, 6989, 7054, 7083, 7153, 7171, 7239, 7252, 7318, 7328, 7388, 7396, 7453, 7461, 7514, 7511, 7564, 7558, 7606, 7598, 7642, 7633, 7671, 7658, 7693, 7678, 7709, 7689]

 ((130, 65), 1, False, True)
(-1, 0) 66 [1, 3, 6, 10, 15, 21, 28, 36, 44, 54, 64, 75, 89, 99, 116, 127, 148, 161, 178, 194, 216, 231, 255, 273, 301, 321, 347, 372, 398, 424, 453, 480, 515, 542, 578, 605, 647, 676, 720, 743, 795, 818,

defaultdict(list,
            {((65, 65), 0, True, True): [1,
              4,
              9,
              14,
              24,
              32,
              41,
              54,
              67,
              90,
              103,
              128,
              148,
              179,
              198,
              233,
              259,
              296,
              328,
              370,
              400,
              445,
              469,
              526,
              547,
              618,
              635,
              714,
              732,
              817,
              837,
              930,
              949,
              1041,
              1070,
              1169,
              1212,
              1303,
              1344,
              1433,
              1480,
              1576,
              1634,
              1730,
              1785,
              1874,
              1949,
              2040,
              2122,
              2210,
 

In [881]:
gardengrowth = defaultdict(list)

maxnr = 0

for tile in sorted(set(gardenopentile.values()), key=lambda x: x[1]):
    if tile[1] > 8:
        break
    if tile[1] < 3:
        continue
    maxnr = max(maxnr, tile[1])

    startingtile, circle, zeror, zeroc = tile
    # print('\n', tile)
    for k,v in gardensize.items():
        if gardenopentile[k] == tile:
            # print(k,gardenopentime[k], v)
            # print(v)
            tst = startingtile, zeror, zeroc
            if tst in gardengrowth and len(gardengrowth[tst]) > 0 and gardengrowth[tst] !=  v:
                print('this is an error since the growth is different', k, tile)
                # pass
            gardengrowth[startingtile, zeror, zeroc] = v
for k, v in gardengrowth.items():
    if len(v) > 1:
        print(v)

[1, 3, 6, 10, 15, 21, 28, 36, 44, 54, 64, 75, 89, 99, 116, 127, 148, 161, 178, 194, 216, 231, 255, 273, 301, 321, 347, 372, 398, 424, 453, 480, 515, 542, 578, 605, 647, 676, 720, 743, 795, 818, 869, 902, 947, 984, 1026, 1071, 1111, 1153, 1202, 1250, 1295, 1342, 1390, 1445, 1495, 1546, 1604, 1657, 1713, 1766, 1827, 1879, 1947, 1997, 2059, 2112, 2174, 2232, 2294, 2350, 2408, 2466, 2526, 2584, 2646, 2703, 2770, 2821, 2888, 2937, 3000, 3054, 3115, 3164, 3229, 3274, 3355, 3385, 3470, 3500, 3590, 3619, 3712, 3733, 3824, 3848, 3938, 3968, 4059, 4095, 4178, 4206, 4291, 4312, 4410, 4434, 4526, 4545, 4634, 4662, 4745, 4780, 4857, 4898, 4971, 5011, 5086, 5127, 5205, 5247, 5321, 5363, 5432, 5475, 5551, 5608, 5685, 5743, 5817, 5873, 5945, 5999, 6056, 6104, 6158, 6207, 6261, 6309, 6357, 6411, 6457, 6501, 6547, 6594, 6637, 6678, 6719, 6759, 6805, 6841, 6886, 6918, 6960, 6991, 7032, 7058, 7105, 7126, 7170, 7188, 7232, 7244, 7286, 7303, 7338, 7351, 7388, 7394, 7437, 7436, 7482, 7475, 7520, 7514, 7553, 

In [882]:
maxnr

8

In [883]:
rows, cols

(131, 131)

In [884]:
edges = {
                           ((0, 0), True, False): 1,
                           ((10, 10), False, True): 1,
                           ((0, 0), False, True): 1,
                           ((10, 10), True, False): 1}
edges


{((0, 0), True, False): 1,
 ((10, 10), False, True): 1,
 ((0, 0), False, True): 1,
 ((10, 10), True, False): 1}

In [885]:
edges = {((0, 65), False, True), ((65, 130), True, False), ((65, 0), True, False), ((130, 65), False, True)}

In [886]:

starttimes = defaultdict(list)
deltas = defaultdict(int)
starts = defaultdict(int)
prev = 0
cnts = defaultdict(lambda: [0,defaultdict(int)])





for a,b in opened:
    new = abs(b)+abs(a)
    if new < prev:
        print('err')
        break
    else:
        cnts[new][0] += 1
        tile = gardenopentile[a,b]
        cnts[new][1][tile] += 1
        prev = new
        starttimes[tile[0], tile[2], tile[3]].append(gardenopentime[a,b])
        if tile[1] == 3:
            starts[tile[0], tile[2], tile[3]] = gardenopentime[a,b]
for k,v in starttimes.items():
    if len(v) >= 4:
        deltas[k] = v[3] - v[2]
        assert deltas[k], v[3] - v[2]
        assert deltas[k], v[4] - v[3]
deltas, starts


(defaultdict(int,
             {((0, 65), False, True): 131,
              ((130, 65), False, True): 131,
              ((65, 0), True, False): 131,
              ((65, 130), True, False): 131,
              ((130, 130), False, False): 131,
              ((0, 130), False, False): 131,
              ((130, 0), False, False): 131,
              ((0, 0), False, False): 131}),
 defaultdict(int,
             {((130, 130), False, False): 263,
              ((0, 130), False, False): 263,
              ((130, 0), False, False): 263,
              ((0, 0), False, False): 263,
              ((65, 130), True, False): 328,
              ((130, 65), False, True): 328,
              ((0, 65), False, True): 328,
              ((65, 0), True, False): 328}))

In [887]:
totalon = 0
totaloff = 0
total = 0
corners = [(0, 0), (0, cols-1), (rows-1, 0), (rows-1, cols-1)]
end = 64
end = 26501365
# end = 5000
def onoff(starttime, duration, amount):
    global totalon, totaloff
    atmax = starttime + duration
    # print('done', starttime, duration, amount, atmax)
    if atmax % 2 == end % 2:
        totalon += amount
    else:
        totaloff += amount
    
# if gardensize[0,0][0] != 1:
#     gardensize[0,0] = [1] + gardensize[0,0]
for i in opened[:13]:
    currentstarttime = gardenopentime[i]
    delta = len(gardensize[i])
    timeleft = end - currentstarttime
    # print(currentstarttime, delta, timeleft, gardensize[i])
    if timeleft >= delta:
        onoff(currentstarttime, delta,1)
    elif timeleft >= 0:
        print('hi', gardensize[i])
        total += gardensize[i][timeleft]
stopped = False
for c in range(1000000000):
    if stopped:
        print('stopped at round', c)
        break
    stopped = True
    
    for node in starts:
        start = starts[node]
        delta = deltas[node]
        currentstarttime = start + delta * c
        if currentstarttime > end:
            continue
        stopped = False
        
        timeleft = end - currentstarttime

        if node in edges:
            # these are the edges, 4 start at identical times, but have different growth. all need to be checked where they are
            # print(start, delta,c, currenttime, timeleft, len(gardengrowth[node]))
            if timeleft >= len(gardengrowth[node]):
                # made it
                onoff(currentstarttime, len(gardengrowth[node]), 1)
            elif timeleft >=0:
                # print('adding')
                total += gardengrowth[node][timeleft]
        else:
            # these are the followers, they may have different times, also have different growth
            if timeleft >= len(gardengrowth[node]):
                # made it
                onoff(currentstarttime, len(gardengrowth[node]), c+2)
                stopped = False
            elif timeleft >=0:
                # print('adding')
                total += (c + 2) * gardengrowth[node][timeleft]
            
totalon * maxx + totaloff * other + total , totalon, totaloff, total

stopped at round 202300


(630661863455116, 40924885401, 40925290000, 6266845599)

In [745]:
gardensize[0,0]

[2, 4, 6, 9, 13, 16, 21, 25, 29, 33, 35, 40, 39]

In [ ]:
1261311984437581 (too high)
630662622690913 (too high)
630661863455116
3314998255866



In [468]:
closedtilesmaxx = sum([maxx for k,v in closedgarden.items() if v % 2 == step % 2])
closedtilesother = sum([other for k,v in closedgarden.items() if v % 2 != step % 2])

closedmaxx = sum([1 for k,v in closedgarden.items() if v % 2 == step % 2])
closedother = sum([1 for k,v in closedgarden.items() if v % 2 != step % 2])

gardentypes = defaultdict(int)

for coor, garden in gardens.items():
    gardentypes[(gardenopentile[coor], (need - gardenopentime[coor]))] += 1
pprint((gardentypes, closedmaxx, closedother))
     
gardengrowth = defaultdict(list)
for tile in sorted(set(gardenopentile.values())):
    for k,v in gardensize.items():
        if gardenopentile[k] == tile:
            gardengrowth[tile] = max(gardengrowth[tile], v, key= lambda x: len(x))
    

# gardentype example = ((10, 10), False, False), with a duration its alive
endpoint = need + 5
from pprint import pprint

for step in range(need, endpoint + 1):
    newgardentypes = defaultdict(int)
    born, dead = 0,0
    for (gardentype, alive), amount in gardentypes.items():
        
        # check if alive is max for growth
        if alive + 1 == len(gardengrowth[gardentype]):
            # todo check if closedmaxx + 1 or closedother
            closedmaxx += amount
            dead += amount
        else:
            newgardentypes[gardentype, alive + 1] += amount
            # check if new babygardens are made
            if len(baby[gardentype][alive]) > 0:
                for newbabytype in baby[gardentype][alive]:
                    newgardentypes[newbabytype, 0] += amount
                    print('born', gardentype, alive, amount, baby[gardentype][alive])
                    born += amount
    print(born, dead)
    gardentypes = newgardentypes
    # print(step)
    # pprint((gardentypes, closedmaxx, closedother))
    
    if step in [need, 6,10,50,64, 100,499, 500,501, 999, 1000,1001, 4999, 5000, 5001] or not step % 1000000:
        total = 0
        for (gardentype, alive), amount in gardentypes.items():
            # print(alive, len(gardengrowth[gardentype]), gardentype)
            total += amount * gardengrowth[gardentype][alive]
        
        print(step, sum([total, closedmaxx * maxx, closedother * other]), total, closedmaxx * maxx, closedother * other)

            
            
        
            
        
        

(defaultdict(<class 'int'>,
             {(((0, 0), False, False), 10): 44,
              (((0, 0), False, True), 4): 1,
              (((0, 0), False, True), 15): 1,
              (((0, 0), True, False), 4): 1,
              (((0, 0), True, False), 15): 1,
              (((0, 10), False, False), 3): 45,
              (((0, 10), False, False), 14): 44,
              (((10, 0), False, False), 3): 45,
              (((10, 0), False, False), 14): 44,
              (((10, 10), False, False), 3): 45,
              (((10, 10), False, False), 14): 44,
              (((10, 10), False, True), 4): 1,
              (((10, 10), True, False), 4): 1,
              (((10, 10), True, False), 15): 1}),
 1936,
 2022)


IndexError: list index out of range

In [441]:
gardengrowth

defaultdict(list,
            {((0, 0), False, False): [1,
              2,
              4,
              5,
              8,
              9,
              12,
              12,
              17,
              18,
              23,
              24,
              29,
              31,
              34,
              35,
              37,
              37,
              41,
              39],
             ((0, 0), False, True): [1,
              2,
              4,
              5,
              9,
              11,
              15,
              16,
              22,
              23,
              29,
              30,
              35,
              35,
              38,
              38,
              41,
              39],
             ((0, 0), True, False): [1,
              2,
              5,
              7,
              11,
              13,
              18,
              18,
              24,
              25,
              29,
              31,
              34,
       

In [254]:
prev = []
cnt = 0
for coor in opened:
    if gardensize[coor] == prev:
        cnt += 1
    else:
        print(cnt, prev)
        prev = gardensize[coor]
        cnt = 1
    

0 []
1 [1, 4, 7, 7, 10, 11, 14, 15, 18, 18, 22, 25, 31, 33, 38, 39]
1 [1, 3, 5, 7, 7, 9, 10, 13, 14, 17, 19, 25, 26, 30, 30, 35, 35, 38, 38, 41, 39]
1 [1, 2, 5, 6, 9, 10, 14, 14, 17, 20, 26, 27, 33, 34, 39, 38]
1 [1, 3, 6, 7, 12, 13, 18, 19, 24, 25, 28, 31, 33, 34, 38, 37, 40, 39]
1 [1, 2, 3, 4, 7, 6, 9, 10, 14, 14, 18, 21, 26, 29, 32, 33, 36, 36, 40, 39]
2 [1, 2, 3, 5, 7, 9, 11, 14, 17, 18, 21, 21, 25, 26, 33, 33, 38, 37, 41, 39]
1 [1, 2, 3, 4, 7, 9, 10, 14, 18, 20, 24, 25, 29, 29, 34, 33, 38, 37, 41, 39]
1 [1, 2, 3, 4, 7, 6, 9, 10, 14, 14, 18, 21, 26, 29, 32, 33, 36, 36, 40, 39]
1 [1, 2, 3, 4, 7, 9, 10, 14, 18, 20, 24, 25, 29, 29, 34, 33, 38, 37, 41, 39]
2 [1, 2, 4, 5, 8, 9, 12, 12, 17, 18, 23, 24, 29, 31, 34, 35, 37, 37, 41, 39]
1 [1, 2, 5, 7, 11, 13, 18, 18, 24, 25, 29, 31, 34, 35, 39, 38, 41, 39]
2 [2, 4, 6, 8, 14, 14, 17, 20, 26, 27, 33, 34, 39, 38]
1 [1, 2, 4, 5, 9, 11, 15, 16, 22, 23, 29, 30, 35, 35, 38, 38, 41, 39]
3 [3, 8, 11, 12, 16, 16, 19, 21, 25, 26, 31, 32, 37, 37, 41, 3

In [161]:
# grid
grid, neigh = gridneigh("t0.txt", to_dict=True, parser=lambda x: [(ch) for ch in x])
# need = 64
maxx = 7717
other = 7693
maxx = 42
other = 39

borders = defaultdict(list)
for k,v in grid.items():
    for wind in [N, S, E, W]:
        new = k[0] + wind[0], k[1] + wind[1]
        if new not in grid:
            new = new[0] % rows, new[1] % cols
            assert new in grid
            if grid[new] != '#':
                borders[k].append((wind, new))



start = [k for k,v in grid.items() if v == 'S'].pop()
valids = {k for k,v in grid.items() if v != '#'}



on = defaultdict(lambda: [0,0])



opengarden = {} # coordinates
closedgarden = dict() # coordinates with a stepmoment
gardens = {(0,0): {start}} # dict with coordinates and a set of cells per garden
need = 500
careful = 0
makers = defaultdict(int)
for step in range(1,need):
    
    newgardens = defaultdict(set)
    oldgarden = gardens.copy()
    for coor, garden in gardens.items():
        for tile in garden:        
            # print(tile)
            for n in neigh[tile]:
                if n in valids:
                    newgardens[coor].add(n)
            for wind, newcoor in borders[tile]:
                # for all bordertiles
                gardenloc = coor[0] + wind[0], coor[1] + wind[1]
                if gardenloc not in closedgarden: # else ignore
                    if gardenloc not in gardens:
                        makers[gardenloc] = step
                        on[gardenloc][0] += 1
                    else:
                        if all(n2 not in oldgarden[gardenloc] for n2 in neigh[newcoor]):
                            print(step, makers[gardenloc], newcoor, oldgarden[gardenloc])
                            careful += 1
                    
                    # print(newcoor)
                    newgardens[gardenloc].add(newcoor)
    gardens = newgardens
    for coor in list(gardens.keys()):
        if len(gardens[coor]) == maxx:
            closedgarden[coor] = step
            del gardens[coor]


    if step in [6,10,50,100,500,1000,5000] or not step % 1000000:
        opentiles = sum([len(v) for k,v in gardens.items()])
        closedtilesmaxx = sum([maxx for k,v in closedgarden.items() if v % 2 == step % 2])
        closedtilesother = sum([other for k,v in closedgarden.items() if v % 2 != step % 2])
        print(step, sum([opentiles, closedtilesmaxx, closedtilesother]), opentiles, closedtilesmaxx, closedtilesother)
        print('open gardens', len(gardens), 'closed gardens', len(closedgarden))
    # print(gardens.keys())
    


6 16 16 0 0
open gardens 1 closed gardens 0
8 7 (7, 10) {(4, 10)}
10 50 50 0 0
open gardens 5 closed gardens 0
11 9 (0, 0) {(5, 0), (4, 1), (3, 0)}
11 9 (10, 0) {(10, 9), (10, 7)}
23 22 (7, 10) {(0, 10), (4, 10), (10, 10)}
24 22 (4, 0) {(1, 0), (0, 1)}
26 22 (10, 0) {(0, 1), (1, 2), (0, 3), (3, 0), (5, 0), (1, 0), (4, 1)}
26 22 (0, 10) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
37 33 (10, 0) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
37 33 (0, 10) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
48 44 (10, 0) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
48 44 (0, 10) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
50 1594 670 378 546
open gardens 34 closed gardens 23
59 55 (10, 0) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
59 55 (0, 10) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
70 66 (10, 0) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
70 66 (0, 10) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
81 77 (10, 0) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
81 77 (0, 10) {(0, 1), (1, 2), (0, 3), (3, 0), (1, 0)}
92 88 (10, 0) {(0

In [158]:
for amount, warnings in on.values():
    if warnings and amount != warnings:
        print(amount, warnings)

1 2
3 1


In [59]:
sum(1 for k,v in grid.items() if v != '#')

81

In [61]:
sum(1 for k,v in grid.items() if v == '#')


40

In [62]:
len(grid)

121

In [104]:
# grid
grid, neigh = gridneigh("t0.txt", to_dict=True, parser=lambda x: [(ch) for ch in x])
need = 150

start = [k for k,v in grid.items() if v == 'S'].pop()
steps = 150
tocheck = deque([start])
seen = set()

for step in range(1,steps):
    for _ in range(len(tocheck)):
        cur = tocheck.popleft()
        for n in neigh[tuple(cur)]:
            if n in grid and grid[n] != '#':
                tocheck.append(n)
    tocheck = set(tocheck)
    tocheck = deque(tocheck)
    print(step, len(tocheck))
len(tocheck)
    
        


1 2
2 4
3 6
4 9
5 13
6 16
7 21
8 25
9 29
10 33
11 35
12 40
13 39
14 42
15 39
16 42
17 39
18 42
19 39
20 42
21 39
22 42
23 39
24 42
25 39
26 42
27 39
28 42
29 39
30 42
31 39
32 42
33 39
34 42
35 39
36 42
37 39
38 42
39 39
40 42
41 39
42 42
43 39
44 42
45 39
46 42
47 39
48 42
49 39
50 42
51 39
52 42
53 39
54 42
55 39
56 42
57 39
58 42
59 39
60 42
61 39
62 42
63 39
64 42
65 39
66 42
67 39
68 42
69 39
70 42
71 39
72 42
73 39
74 42
75 39
76 42
77 39
78 42
79 39
80 42
81 39
82 42
83 39
84 42
85 39
86 42
87 39
88 42
89 39
90 42
91 39
92 42
93 39
94 42
95 39
96 42
97 39
98 42
99 39
100 42
101 39
102 42
103 39
104 42
105 39
106 42
107 39
108 42
109 39
110 42
111 39
112 42
113 39
114 42
115 39
116 42
117 39
118 42
119 39
120 42
121 39
122 42
123 39
124 42
125 39
126 42
127 39
128 42
129 39
130 42
131 39
132 42
133 39
134 42
135 39
136 42
137 39
138 42
139 39
140 42
141 39
142 42
143 39
144 42
145 39
146 42
147 39
148 42
149 39


39

{(3, 4), (4, 3), (4, 5), (5, 4)}

In [6]:
# parse grid with complex numbers
f = open("in.txt").read().split("\n")
lines = [list(line.split()) for line in f]
grid = arr_to_dict(lines, tocomplex=True)
complexneighbors(3+2j)

{(2+2j), (3+1j), (3+3j), (4+2j)}